# **Step#1 : Installing Required Libraries**

In [12]:
!pip3 install transformers==4.34.1
!pip install datasets==2.14.6
!pip install evaluate==0.4.1
!pip3 install protobuf==3.20.3
!pip3 install accelerate==0.24.0
!pip3 install wandb
!pip3 install sentencepiece
!pip3 install seqeval
!wandb disabled ## If you want to log your experiment, remove this line


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=0

# **Step#2 : Download the pre-processed BLURB dataset and FLAX codes**

Here we will download the pre-processed dataset for BLURB benchmark from LinkBERT team.
We need also to download our FLAX code, which is based on Hugginggface pytorch examples and LinkBERT team codes, so all credits for both.

In [3]:
!wget https://nlp.stanford.edu/projects/myasu/LinkBERT/data.zip
!unzip data.zip
!wget https://raw.githubusercontent.com/salrowili/BioM-Transformers/main/misc/run_flax_seq.py
!wget https://raw.githubusercontent.com/salrowili/BioM-Transformers/main/misc/run_flax_ner.py

--2023-11-04 19:59:26--  https://nlp.stanford.edu/projects/myasu/LinkBERT/data.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 458233246 (437M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 437.00M  13.8MB/s    in 21s     

2023-11-04 19:59:47 (20.3 MB/s) - ‘data.zip’ saved [458233246/458233246]

Archive:  data.zip
   creating: data/
   creating: data/tokcls/
   creating: data/tokcls/BC5CDR-disease_hf/
  inflating: data/tokcls/BC5CDR-disease_hf/test.json  
  inflating: data/tokcls/BC5CDR-disease_hf/train.json  
  inflating: data/tokcls/BC5CDR-disease_hf/dev.json  
   creating: data/tokcls/BC2GM_hf/
  inflating: data/tokcls/BC2GM_hf/train.json  
  inflating: data/tokcls/BC2GM_hf/dev.json  
  inflating: data/tokcls/BC2GM_hf/test.json  
   creating: data/tokcls/ebmnlp_hf/
  inflating: data/tok

We need to disable PJRT_DEVICE setting, otherwise the notebook will throw an "Unexpected PJRT_Api size" error. Please make sure to run the cell below everytime you restart this notebook.

In [4]:
import os
# 아직 PJRT를 쓰면 UNEXPECTED PJRT_Api size 오류가 남
os.environ.pop('PJRT_DEVICE', None)
os.environ['XRT_TPU_CONFIG'] = 'localservice;0;localhost:51011'

# **Step#3 : FineTuning BioM-Transformers with FLAX on BLURB dataset**

Here we pick couple of examples with BioM-Transformers on a list of tasks. After each run, the predictions will be saved as BLURB format. After the code is finished, go to the right panel of this Kaggle, and you will see "output" section, showing the saved model and the file that saves the predictions (e.g, BioM-ALBERT-xxlarge_ChemProt.json).

In [10]:
!python3 run_flax_seq.py --model_name_or_path sultan/BioM-ALBERT-xxlarge \
--train_file data/seqcls/chemprot_hf/train.json \
--validation_file data/seqcls/chemprot_hf/dev.json \
--test_file data/seqcls/chemprot_hf/test.json \
--do_train \
--do_eval \
--per_device_train_batch_size 3 \
--per_device_eval_batch_size 8 \
--learning_rate 3e-5 \
--num_train_epochs 6 \
--max_seq_length 256 \
--save_strategy no \
--evaluation_strategy no \
--output_dir out \
--overwrite_output_dir \
--save_strategy no \
--evaluation_strategy no \
--eval_steps 100000 \
--metric_name PRF1 \
--blurb_task ChemProt

/usr/local/lib/python3.8/site-packages/datasets/load.py:2089: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=None' instead.
  warnings.warn(
Extracting data files: 100%|████████████████████| 3/3 [00:00<00:00, 1577.20it/s]
Generating train split: 18035 examples [00:00, 639898.09 examples/s]
Generating validation split: 11268 examples [00:00, 964912.57 examples/s]
Generating test split: 15745 examples [00:00, 913318.44 examples/s]
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--sultan--BioM-ALBERT-xxlarge/snapshots/0ee529e434858b43bbf795853e272a1e77b65bc9/config.json
Model config AlbertConfig {
  "_name_or_path": "sultan/BioM-ALBERT-xxlarge",
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  

 As you can see here, we achieved an F1 score (test_f1) of 80.65 on the chemport task.

In [18]:
!python3 run_flax_ner.py --model_name_or_path sultan/BioM-ELECTRA-Base-Discriminator \
--train_file data/tokcls/BC5CDR-disease_hf/train.json \
--validation_file data/tokcls/BC5CDR-disease_hf/dev.json \
--test_file data/tokcls/BC5CDR-disease_hf/test.json \
--do_train \
--do_eval \
--do_test \
--per_device_train_batch_size 1 \
--learning_rate 7e-5 \
--num_train_epochs 7 \
--output_dir out \
--overwrite_output_dir \
--blurb_task BC5-disease \
--max_seq_length 512 \
--return_entity_level_metrics \
--eval_steps 1000

/usr/local/lib/python3.8/site-packages/datasets/load.py:2089: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=None' instead.
  warnings.warn(
Extracting data files: 100%|████████████████████| 3/3 [00:00<00:00, 1695.12it/s]
Generating train split: 4560 examples [00:00, 242731.47 examples/s]
Generating validation split: 4581 examples [00:00, 416196.05 examples/s]
Generating test split: 4797 examples [00:00, 449209.12 examples/s]
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--sultan--BioM-ELECTRA-Base-Discriminator/snapshots/720be754b9e8a6531556756a5254c52c35e51a06/config.json
Model config ElectraConfig {
  "_name_or_path": "sultan/BioM-ELECTRA-Base-Discriminator",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "finetuning_task": "ner

In [20]:
!python3 run_flax_ner.py --model_name_or_path sultan/BioM-ELECTRA-Base-Discriminator \
--train_file data/tokcls/ebmnlp_hf/train.json \
--validation_file data/tokcls/ebmnlp_hf/dev.json \
--test_file data/tokcls/ebmnlp_hf/test.json \
--do_train \
--do_eval \
--do_test \
--per_device_train_batch_size 1 \
--learning_rate 1e-5 \
--num_train_epochs 1 \
--output_dir out \
--overwrite_output_dir \
--blurb_task "EBM PICO" \
--max_seq_length 512 \
--return_macro_metrics \
--eval_steps 1000

/usr/local/lib/python3.8/site-packages/datasets/load.py:2089: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=None' instead.
  warnings.warn(
Extracting data files: 100%|████████████████████| 3/3 [00:00<00:00, 1635.63it/s]
Generating train split: 40935 examples [00:00, 378016.22 examples/s]
Generating validation split: 10386 examples [00:00, 553998.90 examples/s]
Generating test split: 2076 examples [00:00, 265444.47 examples/s]
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--sultan--BioM-ELECTRA-Base-Discriminator/snapshots/720be754b9e8a6531556756a5254c52c35e51a06/config.json
Model config ElectraConfig {
  "_name_or_path": "sultan/BioM-ELECTRA-Base-Discriminator",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "finetuning_task": "n